<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/MobileNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login

     |████████████████████████████████| 1.9 MB 4.9 MB/s 
     |████████████████████████████████| 166 kB 68.1 MB/s 
     |████████████████████████████████| 182 kB 71.6 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 166 kB 72.2 MB/s 
     |████████████████████████████████| 162 kB 65.8 MB/s 
     |████████████████████████████████| 162 kB 60.6 MB/s 
     |████████████████████████████████| 158 kB 71.8 MB/s 
     |████████████████████████████████| 157 kB 71.9 MB/s 
     |████████████████████████████████| 157 kB 72.3 MB/s 
     |████████████████████████████████| 157 kB 76.0 MB/s 
     |████████████████████████████████| 157 kB 77.9 MB/s 
     |████████████████████████████████| 157 kB 73.4 MB/s 
     |████████████████████████████████| 157 kB 69.3 MB/s 
     |████████████████████████████████| 157 kB 74.3 MB/s 
     |████████████████████████████████| 156 kB 72.9 MB/s 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

class_names = train_ds.class_names
test_class_names = test_ds.class_names
num_classes = len(class_names)
print(class_names)

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 94 files belonging to 5 classes.
['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [12]:
mobilenet = tf.keras.applications.MobileNet(
    input_shape=None,
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling='avg',
    classes=1000,
    classifier_activation="softmax",
)
model_mobilenet=Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  mobilenet,
  #layers.Flatten(),
  #layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
mobilenet.trainable=False
model_mobilenet.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 mobilenet_1.00_224 (Functio  (None, 1024)             3228864   
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 dense_2 (Dense)             (None, 5)                 5125      
                                                                 
Total params: 3,233,989
Trainable params: 5,125
Non-trainable params: 3,228,864
_________________________________________________________________


In [13]:
model_mobilenet.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [14]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "Mobilenet",
)

In [15]:
%%time
epochs=40
history = model_mobilenet.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

Epoch 1/40
22/22 [==============================] - ETA: 0s - loss: 1.0676 - accuracy: 0.6023

wandb: Adding directory to artifact (/content/wandb/run-20221106_040832-1w7j3340/files/model-best)... Done. 0.1s


22/22 [==============================] - 15s 614ms/step - loss: 1.0676 - accuracy: 0.6023 - val_loss: 0.5803 - val_accuracy: 0.7943
Epoch 2/40
22/22 [==============================] - ETA: 0s - loss: 0.4397 - accuracy: 0.8580

wandb: Adding directory to artifact (/content/wandb/run-20221106_040832-1w7j3340/files/model-best)... Done. 0.1s


22/22 [==============================] - 13s 589ms/step - loss: 0.4397 - accuracy: 0.8580 - val_loss: 0.3514 - val_accuracy: 0.8914
Epoch 3/40
22/22 [==============================] - ETA: 0s - loss: 0.2943 - accuracy: 0.9205

...

22/22 [==============================] - 13s 577ms/step - loss: 0.0163 - accuracy: 0.9986 - val_loss: 0.0514 - val_accuracy: 0.9829
Epoch 37/40
22/22 [==============================] - 3s 94ms/step - loss: 0.0165 - accuracy: 1.0000 - val_loss: 0.0519 - val_accuracy: 0.9829
Epoch 38/40
22/22 [==============================] - ETA: 0s - loss: 0.0130 - accuracy: 1.0000

wandb: Adding directory to artifact (/content/wandb/run-20221106_040832-1w7j3340/files/model-best)... Done. 0.1s


22/22 [==============================] - 13s 571ms/step - loss: 0.0130 - accuracy: 1.0000 - val_loss: 0.0501 - val_accuracy: 0.9829
Epoch 39/40
22/22 [==============================] - ETA: 0s - loss: 0.0173 - accuracy: 0.9986

wandb: Adding directory to artifact (/content/wandb/run-20221106_040832-1w7j3340/files/model-best)... Done. 0.1s


22/22 [==============================] - 13s 574ms/step - loss: 0.0173 - accuracy: 0.9986 - val_loss: 0.0488 - val_accuracy: 0.9829
Epoch 40/40
22/22 [==============================] - 3s 94ms/step - loss: 0.0138 - accuracy: 1.0000 - val_loss: 0.0495 - val_accuracy: 0.9829
CPU times: user 6min 48s, sys: 30.4 s, total: 7min 18s
Wall time: 9min 20s


In [16]:
wandb.finish()

accuracy,▁▆▇▇▇███████████████████████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▅▆▆▇▇▇▇▇▇▇▇██▇█████████████████████████
val_loss,█▅▄▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,1.0
best_epoch,38
best_val_loss,0.04881
epoch,39
loss,0.0138
val_accuracy,0.98286


In [11]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4
